In [1]:
from sklearn.metrics import accuracy_score, classification_report
from numpy import mean
from numpy import std
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import model_selection, naive_bayes
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
import pandas as pd
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
import pickle
# from thundersvm import SVC

In [2]:
train_pickle = "../Movies_and_TV_5_balanced_tokenized_with_stopwords.pkl"

In [3]:
def train(df):
    df["reviewText"] = df["reviewText"].apply(lambda x: " ".join(x))
    print("Done joining the text")

    train_x, test_x, train_y, test_y = model_selection.train_test_split(df['reviewText'],df['label'],test_size=0.2)
    print("Done splitting")
    
    tfidf = TfidfVectorizer()
    tfidf.fit(df['reviewText'])
    train_x_tfidf = tfidf.transform(train_x)
    test_x_tfidf = tfidf.transform(test_x)
    print("Done vectorizing")
    
#     svm = LogisticRegression(solver='liblinear', C=1, random_state=42, penalty='l2',max_iter=6000)
    svm = LinearSVC(random_state=42) # 65
#     svm = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0)
#     svm = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None) # 62
#     svm = LogisticRegression(n_jobs=1, C=1e5)
#     svm = SVC()
    svm.fit(train_x_tfidf, train_y)
    print("Done modeling")
    
#     accuracy = svm.score(test_x_tfidf, test_y)
#     print("Accuracy -> ", accuracy * 100, "%")
    
    pred_y = svm.predict(test_x_tfidf)
    print("Accuracy score:", accuracy_score(test_y, pred_y))
    print("Classification report")
    print(classification_report(test_y, pred_y))

    return svm, tfidf

In [4]:
df_train = pd.read_pickle(train_pickle)

In [5]:
import time
start_time = time.time()
svm, tfidf = train(df_train)
print("--- %s seconds ---" % (time.time() - start_time))

Done joining the text
Done splitting
Done vectorizing
Done modeling
Accuracy score: 0.7236
Classification report
              precision    recall  f1-score   support

         0.0       0.72      0.75      0.74     66535
         1.0       0.64      0.59      0.61     66449
         2.0       0.80      0.83      0.82     67016

    accuracy                           0.72    200000
   macro avg       0.72      0.72      0.72    200000
weighted avg       0.72      0.72      0.72    200000

--- 111.5530035495758 seconds ---


In [6]:
with open("../movie-linearsvc-tfidf-alt.pkl", "wb") as handle:
    pickle.dump([svm, tfidf], handle, protocol=pickle.HIGHEST_PROTOCOL)